##**Notebook PC#10 - Part 1**

## Generative Adversarial Networks for the MNIST dataset.

Based on [this content](https://towardsdatascience.com/generative-adversarial-networks-bf4e809180b3), with code corrections.

**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** Ariel Góes de Castro <br>
**Aluno(a):** Francisco Germano Vogt

In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg

2024-05-29 23:30:14.061559: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
class GAN():
    def __init__(self, class_label):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.class_label = class_label

        optimizer = Adam(0.0002, 0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        self.generator = self.build_generator()

        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        self.discriminator.trainable = False

        validity = self.discriminator(img)

        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(256 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 256)))
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):
        (X_train, y_train), (_, _) = cifar10.load_data()

        # Filter the dataset for the specified class label
        X_train = X_train[y_train.flatten() == self.class_label]
        
        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.

        d_losses = []
        d_accuracies = []
        g_losses = []

        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, valid)

            d_losses.append(d_loss[0])
            d_accuracies.append(100 * d_loss[1])
            g_losses.append(g_loss)

            if epoch % 100 == 0:
                print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))

            if epoch % sample_interval == 0:
                self.sample_images(epoch)

        return d_losses, d_accuracies, g_losses

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, :])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig("cifar10_class_%d_epoch_%d.png" % (self.class_label, epoch))
        plt.close()

In [3]:
# Example: Train the GAN on class 3 (cats) of CIFAR-10
gan = GAN(class_label=3)
d_losses, d_accuracies, g_losses = gan.train(epochs=30001, batch_size=128, sample_interval=1000)

2024-05-29 23:30:16.342070: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-05-29 23:30:16.343210: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-05-29 23:30:16.358205: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-29 23:30:16.358229: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ariel-Nitro-AN515-44): /proc/driver/nvidia/version does not exist
2024-05-29 23:30:16.358808: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 9, 9, 64)          0         
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 64)          256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 9, 9, 64)          0

2024-05-29 23:30:17.208102: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-05-29 23:30:17.211440: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2894720000 Hz


0 [D loss: 1.182238, acc.: 33.98%] [G loss: 0.668391]
100 [D loss: 0.001844, acc.: 100.00%] [G loss: 0.067599]
200 [D loss: 0.001041, acc.: 100.00%] [G loss: 0.068553]
300 [D loss: 0.007705, acc.: 100.00%] [G loss: 0.243691]
400 [D loss: 0.001500, acc.: 100.00%] [G loss: 0.086267]
500 [D loss: 0.000195, acc.: 100.00%] [G loss: 0.168551]
600 [D loss: 0.000087, acc.: 100.00%] [G loss: 0.054163]
700 [D loss: 0.000078, acc.: 100.00%] [G loss: 0.044470]
800 [D loss: 0.000121, acc.: 100.00%] [G loss: 0.050589]
900 [D loss: 0.000105, acc.: 100.00%] [G loss: 0.048611]
1000 [D loss: 0.000026, acc.: 100.00%] [G loss: 0.034474]
1100 [D loss: 0.000023, acc.: 100.00%] [G loss: 0.029668]
1200 [D loss: 0.000031, acc.: 100.00%] [G loss: 0.036412]
1300 [D loss: 0.000020, acc.: 100.00%] [G loss: 0.043002]
1400 [D loss: 0.000031, acc.: 100.00%] [G loss: 0.046770]
1500 [D loss: 0.000032, acc.: 100.00%] [G loss: 0.091421]
1600 [D loss: 0.000013, acc.: 100.00%] [G loss: 0.034860]
1700 [D loss: 0.000026, acc

: 

In [ ]:
# Visualize losses
plt.plot(d_losses, label='Discriminator')
plt.plot(g_losses, label='Generator')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
# Visualize accuracy
plt.plot(d_accuracies)
plt.xlabel('Epochs')
plt.ylabel('Accuracy of the Discriminator')
plt.show()